# Evaluation of a Pipeline and its Components

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial5_Evaluation.ipynb)

To be able to make a statement about the quality of results a question-answering pipeline or any other pipeline in haystack produces, it is important to evaluate it. Furthermore, evaluation allows determining which components of the pipeline can be improved.
The results of the evaluation can be saved as CSV files, which contain all the information to calculate additional metrics later on or inspect individual predictions.

### Prepare environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/master/docs/_src/img/colab_gpu_runtime.jpg">

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

In [ ]:
from haystack.modeling.utils import initialize_device_settings
devices, n_gpu = initialize_device_settings(use_cuda=True)

## Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (eg., in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [ ]:
# If Docker is available: Start Elasticsearch as docker container
# from haystack.utils import launch_es
# launch_es()

# Alternative in Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

## Fetch, Store And Preprocess the Evaluation Dataset

In [ ]:
from haystack.utils import fetch_archive_from_http

# Download evaluation data, which is a subset of Natural Questions development set containing 50 documents with one question per document and multiple annotated answers
doc_dir = "../data/nq"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/nq_dev_subset_v2.json.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

In [ ]:
# make sure these indices do not collide with existing ones, the indices will be wiped clean before data is inserted
doc_index = "tutorial5_docs"
label_index = "tutorial5_labels"

In [ ]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index=doc_index,
                                            label_index=label_index, embedding_field="emb",
                                            embedding_dim=768, excluded_meta_data=["emb"])

In [ ]:
from haystack.nodes import PreProcessor

# Add evaluation data to Elasticsearch Document Store
# We first delete the custom tutorial indices to not have duplicate elements
# and also split our documents into shorter passages using the PreProcessor
preprocessor = PreProcessor(
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.
document_store.add_eval_data(
    filename="../data/nq/nq_dev_subset_v2.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor
)

## Initialize the Two Components of an ExtractiveQAPipeline: Retriever and Reader

In [ ]:
# Initialize Retriever
from haystack.nodes import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)
# Alternative: Evaluate dense retrievers (DensePassageRetriever or EmbeddingRetriever)
# DensePassageRetriever uses two separate transformer based encoders for query and document.
# In contrast, EmbeddingRetriever uses a single encoder for both.
# Please make sure the "embedding_dim" parameter in the DocumentStore above matches the output dimension of your models!
# Please also take care that the PreProcessor splits your files into chunks that can be completely converted with
#        the max_seq_len limitations of Transformers
# The SentenceTransformer model "all-mpnet-base-v2" generally works well with the EmbeddingRetriever on any kind of English text.
# For more information check out the documentation at: https://www.sbert.net/docs/pretrained_models.html
# from haystack.retriever import DensePassageRetriever, EmbeddingRetriever
# retriever = DensePassageRetriever(document_store=document_store,
#                                   query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#                                   passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#                                   use_gpu=True,
#                                   max_seq_len_passage=256,
#                                   embed_title=True)
# retriever = EmbeddingRetriever(document_store=document_store, model_format="sentence_transformers",
#                                embedding_model="all-mpnet-base-v2")
# document_store.update_embeddings(retriever, index=doc_index)

In [ ]:
# Initialize Reader
from haystack.nodes import FARMReader

reader = FARMReader("deepset/roberta-base-squad2", top_k=4, return_no_answer=True)

# Define a pipeline consisting of the initialized retriever and reader
from haystack.pipelines import ExtractiveQAPipeline
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# The evaluation also works with any other pipeline.
# For example you could use a DocumentSearchPipeline as an alternative:
# from haystack.pipelines import DocumentSearchPipeline
# pipeline = DocumentSearchPipeline(retriever=retriever)

## Evaluation of an ExtractiveQAPipeline
Here we evaluate retriever and reader in open domain fashion on the full corpus of documents i.e. a document is considered
correctly retrieved if it contains the gold answer string within it. The reader is evaluated based purely on the
predicted answer string, regardless of which document this came from and the position of the extracted span.

The generation of predictions is seperated from the calculation of metrics. This allows you to run the computation-heavy model predictions only once and then iterate flexibly on the metrics or reports you want to generate.


In [ ]:
from haystack.schema import EvaluationResult, MultiLabel

# We can load evaluation labels from the document store
eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=False)

# Alternative: Define queries and labels directly
# from haystack.schema import Answer, Document, Label, Span
# eval_labels = [
#        MultiLabel(labels=[Label(query="who is written in the book of life",
#        answer=Answer(answer="every person who is destined for Heaven or the World to Come",
#        offsets_in_context=[Span(374, 434)]),
#        document=Document(id='1b090aec7dbd1af6739c4c80f8995877-0',
#        content_type="text",
#        content='Book of Life - wikipedia Book of Life Jump to: navigation, search This article is about the book mentioned in Christian and Jewish religious teachings. For other uses, see The Book of Life. In Christianity and Judaism, the Book of Life (Hebrew: ספר החיים, transliterated Sefer HaChaim; Greek: βιβλίον τῆς ζωῆς Biblíon tēs Zōēs) is the book in which God records the names of every person who is destined for Heaven or the World to Come. According to the Talmud it is open on Rosh Hashanah, as is its analog for the wicked, the Book of the Dead. For this reason extra mention is made for the Book of Life during Amidah recitations during the Days of Awe, the ten days between Rosh Hashanah, the Jewish new year, and Yom Kippur, the day of atonement (the two High Holidays, particularly in the prayer Unetaneh Tokef). Contents (hide) 1 In the Hebrew Bible 2 Book of Jubilees 3 References in the New Testament 4 The eschatological or annual roll-call 5 Fundraising 6 See also 7 Notes 8 References In the Hebrew Bible(edit) In the Hebrew Bible the Book of Life - the book or muster-roll of God - records forever all people considered righteous before God'),
#        is_correct_answer=True,
#        is_correct_document=True,
#        origin="gold-label")])
#    ]

# Similar to pipeline.run() we can execute pipeline.eval()
eval_result = pipeline.eval(
    labels=eval_labels,
    params={"Retriever": {"top_k": 5}}
)

In [9]:
# The EvaluationResult contains a pandas dataframe for each pipeline node.
# That's why there are two dataframes in the EvaluationResult of an ExtractiveQAPipeline.

retriever_result = eval_result["Retriever"]
retriever_result.head()

,content,document_id,type,gold_document_ids,gold_document_contents,gold_id_match,answer_match,gold_id_or_answer_match,rank,node,query
0,"people considered righteous before God. God has such a book, and to be blott...",1b090aec7dbd1af6739c4c80f8995877-1,document,"[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]","[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,1,Retriever,who is written in the book of life
1,"as adversaries (of God). Also, according to ib. xxxvi. 10, one who contrives...",1b090aec7dbd1af6739c4c80f8995877-2,document,"[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]","[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,2,Retriever,who is written in the book of life
2,the citizens' registers. The life which the righteous participate in is to b...,1b090aec7dbd1af6739c4c80f8995877-6,document,"[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]","[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,3,Retriever,who is written in the book of life
3,"apostles' names are ``written in heaven'' (Luke x. 20), or ``the fellow-work...",1b090aec7dbd1af6739c4c80f8995877-3,document,"[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]","[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,4,Retriever,who is written in the book of life
4,The Absolutely True Diary of a Part-Time Indian - wikipedia The Absolutely T...,e9260cbbc129f4246ee8fcfbbe385822-0,document,"[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]","[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,5,Retriever,who is written in the book of life


In [10]:
reader_result = eval_result["Reader"]
reader_result.head()

,answer,document_id,offsets_in_document,context,type,gold_answers,gold_offsets_in_documents,gold_document_ids,exact_match,f1,rank,node,query
0,,None,"[{'start': 0, 'end': 0}]",None,answer,"[all people considered righteous before God, every person who is destined fo...","[[Span(start=1107, end=1149)], [Span(start=374, end=434)]]","[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]",0.0,0.000000,1,Reader,who is written in the book of life
1,those whose names are written in the Book of Life from the foundation of the...,1b090aec7dbd1af6739c4c80f8995877-3,"[{'start': 576, 'end': 658}]","ohn of Patmos. As described, only those whose names are written in the Book ...",answer,"[all people considered righteous before God, every person who is destined fo...","[[Span(start=1107, end=1149)], [Span(start=374, end=434)]]","[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]",0.0,0.083333,2,Reader,who is written in the book of life
2,only the names of the righteous,1b090aec7dbd1af6739c4c80f8995877-1,"[{'start': 498, 'end': 529}]",. The Psalmist likewise speaks of the Book of Life in which only the names o...,answer,"[all people considered righteous before God, every person who is destined fo...","[[Span(start=1107, end=1149)], [Span(start=374, end=434)]]","[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]",0.0,0.200000,3,Reader,who is written in the book of life
3,those who are found written in the book and who shall escape the troubles pr...,1b090aec7dbd1af6739c4c80f8995877-6,"[{'start': 135, 'end': 305}]",those who are found written in the book and who shall escape the troubles pr...,answer,"[all people considered righteous before God, every person who is destined fo...","[[Span(start=1107, end=1149)], [Span(start=374, end=434)]]","[1b090aec7dbd1af6739c4c80f8995877-0, 1b090aec7dbd1af6739c4c80f8995877-0]",0.0,0.111111,4,Reader,who is written in the book of life
0,her cousin,965a125f65658579529b39f8e4344969-3,"[{'start': 423, 'end': 433}]",ng a story in the newspaper of a 12-year-old girl getting pregnant by her co...,answer,[Ethel ``Edy'' Proctor],"[[Span(start=181, end=202)]]",[965a125f65658579529b39f8e4344969-3],0.0,0.000000,1,Reader,who was the girl in the video brenda got a baby


In [ ]:
# We can filter for all documents retrieved for a given query
retriever_book_of_life = retriever_result[retriever_result['query'] == "who is written in the book of life"]

In [ ]:
# We can also filter for all answers predicted for a given query
reader_book_of_life = reader_result[reader_result['query'] == "who is written in the book of life"]

In [ ]:
# Save the evaluation result so that we can reload it later and calculate evaluation metrics without running the pipeline again.
eval_result.save("../")

## Calculating Evaluation Metrics
Load an EvaluationResult to quickly calculate standard evaluation metrics for all predictions, such as F1-score of each individual prediction of the Reader node or recall of the retriever.

In [10]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 0.38
Retriever - Recall (multiple relevant documents): 0.38
Retriever - Mean Reciprocal Rank: 0.23433333333333334
Retriever - Precision: 0.07600000000000001
Retriever - Mean Average Precision: 0.23433333333333334
Reader - F1-Score: 0.7578128538128539
Reader - Exact Match: 0.72


## Generating an Evaluation Report
A summary of the evaluation results can be printed to get a quick overview. It includes some aggregated metrics and also shows a few wrongly predicted examples.

In [15]:
pipeline.print_eval_report(saved_eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit:  0.42
                        | recall_single_hit_top_1:  0.42
                        |
                      Reader
                        |
                        | exact_match:  0.72
                        | exact_match_top_1:  0.48
                        | f1: 0.758
                        | f1_top_1:   0.5
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	who is written in the book of life
Gold Document Ids: 
 	1b090aec7dbd1af6739c4c80f8995877-0
 	1b090aec7dbd1af6739c4c80f8995877-0
Metrics: 
 	recall_multi_hit: 0.0
 	recall_single_hit: 0.0
 	precision: 0.0
 	map: 0.0
 	mrr: 0.0
Documents: 
 	content: people considered righteous before God. God has s

## Advanced Evaluation Metrics
As an advanced evaluation metric, semantic answer similarity (SAS) can be calculated. This metric takes into account whether the meaning of a predicted answer is similar to the annotated gold answer rather than just doing string comparison.
To this end SAS relies on pre-trained models. For English, we recommend "cross-encoder/stsb-roberta-large", whereas for German we recommend "deepset/gbert-large-sts". A good multilingual model is "sentence-transformers/paraphrase-multilingual-mpnet-base-v2".
More info on this metric can be found in our [paper](https://arxiv.org/abs/2108.06130) or in our [blog post](https://www.deepset.ai/blog/semantic-answer-similarity-to-evaluate-qa).

In [ ]:
advanced_eval_result = pipeline.eval(
        labels=eval_labels,
        params={"Retriever": {"top_k": 1}},
        sas_model_name_or_path="cross-encoder/stsb-roberta-large"
    )

metrics = advanced_eval_result.calculate_metrics()
print(metrics["Reader"]["sas"])

## Evaluation of Individual Components: Retriever
Here we evaluate only the retriever, based on whether the gold_label document is retrieved.

In [ ]:
## Evaluate Retriever on its own
retriever_eval_results = retriever.eval(top_k=5, label_index=label_index, doc_index=doc_index)
# Retriever Recall is the proportion of questions for which the correct document containing the answer is
# among the correct documents
print("Retriever Recall:", retriever_eval_results["recall"])
# Retriever Mean Avg Precision rewards retrievers that give relevant documents a higher rank
print("Retriever Mean Avg Precision:", retriever_eval_results["map"])

## Evaluation of Individual Components: Reader
Here we evaluate only the reader in a closed domain fashion i.e. the reader is given one query
and its corresponding relevant document and metrics are calculated on whether the right position in this text is selected by
the model as the answer span (i.e. SQuAD style)

In [ ]:
# Evaluate Reader on its own
reader_eval_results = reader.eval(document_store=document_store, device=devices[0], label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
#reader_eval_results = reader.eval_on_file("../data/nq", "nq_dev_subset_v2.json", device=device)

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
# Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
# Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)
- [FARM](https://github.com/deepset-ai/FARM)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Slack](https://haystack.deepset.ai/community/join) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs)